In [1]:
import spacy
import pandas as pd

data = pd.read_csv("./datasets/nova_acrop.csv")
data.size

3674

A base de dados foi gerada pela query:

```
union customEvents
| where name == "MessageIn" and customDimensions ["IntentName"] == "unknown"
| order by timestamp
| project Messages = parse_json(tostring(customDimensions["Message"])).text
| where Messages  != ""
```

Ou seja, todas as frases de intenção desconhecida

O algoritmo de frequencia considera o tipo sintatico da palavra, além do que desconsidera palavras irrelevantes e remove plurais.

In [2]:
nlp = spacy.load("pt_core_news_sm")
vectors = []
for message in data.Messages:
    doc = nlp(message)
    vectors.append(doc.vector)

data["Vectors"] = pd.Series(vectors)

In [23]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=20, random_state=0).fit(vectors)

In [24]:
data["Classe"] = pd.Series()
data["Similarity"] = pd.Series()

<ipython-input-24-fb9872cc76bc>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data["Classe"] = pd.Series()
<ipython-input-24-fb9872cc76bc>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data["Similarity"] = pd.Series()


In [36]:
import numpy as np

doc = nlp("Preciso de informação")
vec = doc.vector

def cossine(vec1,vec2):
    return np.dot(vec1/np.linalg.norm(vec1),vec2/np.linalg.norm(vec2))

def nearest(word_vector, data):
    nearest = None
    nearestSim = 0
    for i in range(data.shape[0]):
        try:
            sim = cossine(word_vector,data.iloc[i].Vectors)
            if(sim > nearestSim):
                nearest = data.iloc[i]
                nearestSim = sim
        except:
            print(i)
    return nearest,nearestSim,i

def nearest_center(word_vector, centers):
    nearest = None
    nearestSim = 0
    index = 0
    for i in range(centers.shape[0]):
        sim = cossine(word_vector,centers[i])
        if(sim > nearestSim):
            nearest = centers[i]
            nearestSim = sim
            index = i
    return nearest,nearestSim,index

def defineClasses(data,kmeans):
    for i in range(data.shape[0]):
        nearest,sim,index = nearest_center(data.iloc[i].Vectors, kmeans.cluster_centers_)
        data.loc[i].Classe = index
        data.loc[i].Similarity = sim
        
defineClasses(data,kmeans)

In [47]:
result = data[data.iloc[data.Classe.sort_values().index].Similarity > .8]
result = result.drop("Vectors", axis=1)
result.to_csv("./resultado.csv")
result.head()

<ipython-input-47-1b70784eb63c>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = data[data.iloc[data.Classe.sort_values().index].Similarity > .8]


,Messages,Classe,Similarity
1,Ok,4.0,0.999993
2,Agradeço.,16.0,0.801845
8,?,6.0,0.990349
11,Oi,3.0,0.999918
13,52365859615,13.0,0.861242


In [25]:
nlp = spacy.load("pt_core_news_sm")
"""
unidecode remove acentos
"""    
import unidecode

wordFrequency = {}
def computeFrequency(wordFrequency, nlp, data, wordType = ['NOUN']):
    for string in data.Messages:
        string = string.lower()
        doc = nlp(string)
        for token in doc:
            if token.is_stop:
                continue
            if token.pos_ in wordType:
                text = token.text
                text = unidecode.unidecode(text)
                if len(text) == 0:
                    continue
                if text[-1] == "s":
                    text = text[:-1]
                if text in wordFrequency.keys():
                    wordFrequency[text]+=1
                else:
                    wordFrequency[text]=1


In [19]:
computeFrequency(wordFrequency,nlp,data)

### Número Máximo de Palavras

Para alterar o número máximo de palavras altere a variável a seguir:

In [20]:
max_words = 200

### Núvem sem aplicar limpeza 

A primeira demonstração é baseada no texto todo, sem aplicar nenhum modelo de NLP para identificar os tipos sintáticos. O resultado foi bastante poluido, além de encontrar termos concatenados.

In [21]:
from wordcloud import WordCloud

text = ''
for phrase in data.Messages:
    text += phrase + "\n"
wordcloud = WordCloud().generate(text)

import matplotlib.pyplot as plt

wordcloud = WordCloud(background_color="white",max_words=max_words,max_font_size=40, relative_scaling=.5).generate(text)
plt.figure(figsize=(100,100), dpi = 200)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

### Núvem de substantivos

In [22]:
wordcloud = WordCloud(background_color="white",max_words=max_words,max_font_size=50, relative_scaling=.5).generate_from_frequencies(wordFrequency)
plt.figure(figsize=(100,100), dpi = 200)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

### Núvem de Verbos

In [23]:
wordFrequencyTest = {}
computeFrequency(wordFrequencyTest,nlp,data.head(5000), ['VERB'])

In [24]:
wordcloud = WordCloud(background_color="white",max_words=max_words,max_font_size=40, relative_scaling=.5).generate_from_frequencies(wordFrequencyTest)
plt.figure(figsize=(100,100), dpi = 200)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()